In [7]:
import mediapipe as mp
import cv2
import os
import matplotlib.pyplot as plt
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = HandLandmarkerOptions(
    base_options = BaseOptions(model_asset_path="hand_landmarker.task"),
    running_mode = VisionRunningMode.IMAGE
)

detector = vision.HandLandmarker.create_from_options(options)


subdirectories = ["./label_images/hello_label", "./label_images/i_love_you_label", "./label_images/yes_label"]
label_number = {"hello_label": 0, "i_love_you_label": 1, "yes_label": 2}

print(subdirectories[0].split("/"))

data = []
labels = []

for directory in subdirectories:
    print("***IN DIRECTORY: " + directory)
    print(" ")
    for filename in os.listdir(directory):
        if filename !=".ipynb_checkpoints":
            data_to_append_from_file = []
            image = mp.Image.create_from_file(directory + "/" + filename)
            results = detector.detect(image)
            
            if len(results.hand_landmarks) > 0: #if it detected a hand
                for hand_points in results.hand_landmarks[0]: #all the hand point landmarks in the first hand found
                    data_to_append_from_file.append([hand_points.x, hand_points.y, hand_points.z])
                    
                data.append(data_to_append_from_file)
                labels.append(label_number[directory.split("/")[2]])
#                 labels.append(directory.split("/")[2])

data = np.array(data)

labels = np.array(labels)

from tensorflow.keras.utils import to_categorical

X = data
y = labels

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

y_train_categorical = to_categorical(y_train, 3)
y_test_categorical = to_categorical(y_test, 3)

#Model creation

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()

model.add(Dense(21, activation="relu"))

model.add(Dense(10, activation="relu"))

model.add(Dense(21, activation="relu"))

model.add(Flatten()) #3d ---> 2d

model.add(Dense(3, activation="softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

early_stop = EarlyStopping(monitor="loss", patience=5, mode="min", verbose=1)

model.fit(X_train, y_train_categorical, validation_data=(X_test, y_test_categorical), epochs=450, callbacks=[early_stop])

model.save("sign_language_model.keras")

['.', 'label_images', 'hello_label']
***IN DIRECTORY: ./label_images/hello_label
 
***IN DIRECTORY: ./label_images/i_love_you_label
 
***IN DIRECTORY: ./label_images/yes_label
 
Epoch 1/450
8/8 [==============================] - 1s 18ms/step - loss: 1.0835 - val_loss: 1.0577
Epoch 2/450
8/8 [==============================] - 0s 4ms/step - loss: 1.0346 - val_loss: 1.0147
Epoch 3/450
8/8 [==============================] - 0s 4ms/step - loss: 0.9895 - val_loss: 0.9694
Epoch 4/450
8/8 [==============================] - 0s 3ms/step - loss: 0.9496 - val_loss: 0.9379
Epoch 5/450
8/8 [==============================] - 0s 3ms/step - loss: 0.9113 - val_loss: 0.9082
Epoch 6/450
8/8 [==============================] - 0s 3ms/step - loss: 0.8813 - val_loss: 0.8812
Epoch 7/450
8/8 [==============================] - 0s 3ms/step - loss: 0.8563 - val_loss: 0.8642
Epoch 8/450
8/8 [==============================] - 0s 3ms/step - loss: 0.8357 - val_loss: 0.8397
Epoch 9/450
8/8 [============================

Epoch 83/450
8/8 [==============================] - 0s 3ms/step - loss: 0.0374 - val_loss: 0.0433
Epoch 84/450
8/8 [==============================] - 0s 3ms/step - loss: 0.0389 - val_loss: 0.0647
Epoch 85/450
8/8 [==============================] - 0s 3ms/step - loss: 0.0434 - val_loss: 0.0479
Epoch 86/450
8/8 [==============================] - 0s 3ms/step - loss: 0.0497 - val_loss: 0.0609
Epoch 87/450
8/8 [==============================] - 0s 3ms/step - loss: 0.0404 - val_loss: 0.0467
Epoch 88/450
8/8 [==============================] - 0s 3ms/step - loss: 0.0419 - val_loss: 0.0512
Epoch 00088: early stopping
